In [1]:
import pandas as pd
import numpy as np
import lightgbm 
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

In [2]:
#loading files
test=pd.read_csv("/kaggle/input/prml-data-contest-nov-2020/test.csv")
train=pd.read_csv("/kaggle/input/prml-data-contest-nov-2020/train.csv")
biken=pd.read_csv("/kaggle/input/prml-data-contest-nov-2020/bikers_network.csv")
bikers=pd.read_csv("/kaggle/input/prml-data-contest-nov-2020/bikers.csv")
tours=pd.read_csv("/kaggle/input/prml-data-contest-nov-2020/tours.csv")
tourco=pd.read_csv("/kaggle/input/prml-data-contest-nov-2020/tour_convoy.csv")
# cleaning data for tour convoy
tourco['not_going'].fillna("No", inplace = True)  
tourco['maybe'].fillna("No", inplace = True)  
tourco['going'].fillna("No", inplace = True)  
tourco['invited'].fillna("No", inplace = True) 
# cleaning data for bikers
bikers['gender'].fillna("No Gender", inplace = True)
bikers['location_id'].fillna('NO',inplace= True)
bikers['language_id'].fillna('NO',inplace= True)
bikers['bornIn'].replace(to_replace=['None'], value=np.nan, inplace=True)
bikers['bornIn'].fillna(bikers['bornIn'].mode()[0], inplace=True)
def fx(x):
    try:
        return int(x)
    except:
        return bikers['bornIn'].mode()[0]
bikers['bornIn']=bikers['bornIn'].apply(lambda x: fx(x))
# cleaning data for bikers network 
biken['friends'].fillna("No", inplace = True)  

In [ ]:
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 
longitude1 = {}
latitude1 = {} 
def findGeocode(city): 
    try: 
        
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="your_app_name") 
        
        return geolocator.geocode(city,timeout=1000) 
    
    except GeocoderTimedOut: 
        
        return findGeocode(city)


# each value from city column 
# will be fetched and sent to 
# function find_geocode 
counti=0
for i in bikers["area"]:
    if (counti%1000)==0:
        print(i,counti,len(bikers['area']))
    counti+=1
    if i  in latitude1:
        continue
    if findGeocode(i) != None: 
        loc = findGeocode(i) 
        # coordinates returned from 
        # function is stored into 
        # two separate list 
        latitude1[i] = loc.latitude 
        longitude1[i] = loc.longitude 


    else: 
        latitude1[i] = np.nan 
        longitude1[i] = np.nan 

In [ ]:
bikeloc =pd.DataFrame(columns=["lat","long"])
for i in range(bikers.shape[0]):
    bikeloc.at[i,'lat']=latitude1[bikers.at[i,'area']]
    bikeloc.at[i,'long']=longitude1[bikers.at[i,'area']]


In [ ]:
bikeloc['lat'].fillna(bikeloc['lat'].mode()[0], inplace=True)
bikeloc['long'].fillna(bikeloc['long'].mode()[0], inplace=True)
tours['longitude'].fillna(tours['longitude'].mode()[0], inplace=True)
tours['latitude'].fillna(tours['latitude'].mode()[0], inplace=True)

In [ ]:
# Python 3 program to calculate Distance Between Two Points on Earth 
from math import radians, cos, sin, asin, sqrt 
def distance1(lat1, lat2, lon1, lon2):
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
    
    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    c = 2 * asin(sqrt(a)) 
    
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
    
    # calculate the result 
    return(c * r) 

In [ ]:
#born data for ourtrain and ourtest
dicborn=dict(zip(bikers['biker_id'],bikers['bornIn']))
born=[]
born1=[]
for i in range(train.shape[0]):
    born.append(dicborn[train.at[i,'biker_id']])
    
for i in range(test.shape[0]):
    born1.append(dicborn[test.at[i,'biker_id']])

    

In [ ]:
#if friend is organizing tour,data for ourtrain and ourtest
dicfriend=dict(zip(biken['biker_id'],biken['friends']))
dicorg=dict(zip(tours['tour_id'],tours['biker_id']))
ifri=[]
ifri1=[]
for i in range(train.shape[0]):
    b=train.at[i,'biker_id']
    t=train.at[i,'tour_id']
    org=dicorg[t]
    
    if (dicfriend[b].find(org)== -1) :
        ifri.append(0)
    else :
        ifri.append(10)
for i in range(test.shape[0]):
    b=test.at[i,'biker_id']
    t=test.at[i,'tour_id']
    org=dicorg[t]
    
    if (dicfriend[b].find(org)== -1) :
        ifri1.append(0)
    else :
        ifri1.append(10)

In [ ]:
#male female data for ourtrain and ourtest
male=[]
female=[]
dicgen=dict(zip(bikers['biker_id'],bikers['gender']))
for i in range(train.shape[0]):
    if dicgen[train.at[i,'biker_id']]=='male':
        male.append(1)
        female.append(0)
    elif dicgen[train.at[i,'biker_id']]=='female':
        male.append(0)
        female.append(1)
    else:
        male.append(0)
        female.append(0)
male1=[]
female1=[]
for i in range(test.shape[0]):
    if dicgen[test.at[i,'biker_id']]=='male':
        male1.append(1)
        female1.append(0)
    elif dicgen[test.at[i,'biker_id']]=='female':
        male1.append(0)
        female1.append(1)
    else:
        male1.append(0)
        female1.append(0)

In [ ]:
#timegap for ourtrain, ourtest
from datetime import date
def numOfDays(date1, date2):
    return (date2-date1).days
dic = dict(zip(tours['tour_id'],tours['tour_date']))
timegap = []
timegap1 = []
for i in range(0,train.shape[0]):
    t = dic[train.at[i,"tour_id"]]
    ts = train.at[i,"timestamp"]
    d1 = int(t[0:2])
    m1 = int(t[3:5])
    y1 = int(t[6:10])
    d2 = int(ts[0:2])
    m2 = int(ts[3:5])
    y2 = int(ts[6:10])
    date1 = date(y1,m1,d1)
    date2 = date(y2,m2,d2)
    timegap.append(numOfDays(date1,date2))
for i in range(0,test.shape[0]):
    t = dic[test.at[i,"tour_id"]]
    ts = test.at[i,"timestamp"]
    d1 = int(t[0:2])
    m1 = int(t[3:5])
    y1 = int(t[6:10])
    d2 = int(ts[0:2])
    m2 = int(ts[3:5])
    y2 = int(ts[6:10])
    date1 = date(y1,m1,d1)
    date2 = date(y2,m2,d2)
    timegap1.append(numOfDays(date1,date2))

In [ ]:
# w's,la2,lo2,lo1,la1,friends invited,maybe,going,not going,distance  for ourtrain
w1=[]
w2=[]
w3=[]
w4=[]
w5=[]
w6=[]
w7=[]
w8=[]
w9=[]
w10=[]
w11=[]
w12=[]
w13=[]
w14=[]
w15=[]
w16=[]
w17=[]
w18=[]
w19=[]
w20=[]
w21=[]
w22=[]
w23=[]
w24=[]
w25=[]
w26=[]
w27=[]
w28=[]
w29=[]
w30=[]
w31=[]
w32=[]
w33=[]
w34=[]
w35=[]
w36=[]
w37=[]
w38=[]
w39=[]
w40=[]
w41=[]
w42=[]
w43=[]
w44=[]
w45=[]
w46=[]
w47=[]
w48=[]
w49=[]
w50=[]
w51=[]
w52=[]
w53=[]
w54=[]
w55=[]
w56=[]
w57=[]
w58=[]
w59=[]
w60=[]
w61=[]
w62=[]
w63=[]
w64=[]
w65=[]
w66=[]
w67=[]
w68=[]
w69=[]
w70=[]
w71=[]
w72=[]
w73=[]
w74=[]
w75=[]
w76=[]
w77=[]
w78=[]
w79=[]
w80=[]
w81=[]
w82=[]
w83=[]
w84=[]
w85=[]
w86=[]
w87=[]
w88=[]
w89=[]
w90=[]
w91=[]
w92=[]
w93=[]
w94=[]
w95=[]
w96=[]
w97=[]
w98=[]
w99=[]
w100=[]
wother = []
la2 = []
lo2 = []
la1 = []
lo1 = []
invited = []
notgoing = []
maybe = []
going = []
distance = []
dic1=dict(zip(tourco['tour_id'],tourco['not_going']))
dic2=dict(zip(tourco['tour_id'],tourco['maybe']))
dic3=dict(zip(tourco['tour_id'],tourco['going']))
dic4=dict(zip(tourco['tour_id'],tourco['invited']))

for i in range(train.shape[0]):
    print(i)
    b = train.at[i,'biker_id']
    l = bikers.index[bikers['biker_id'] == b].tolist()
    idxi1 = l[0]
    la1.append(bikeloc.at[idxi1,"lat"])
    lo1.append(bikeloc.at[idxi1,"long"])
    t = train.at[i,'tour_id']
    l = tours.index[tours['tour_id'] == t].tolist()
    idxi = l[0]
    la2.append(tours.at[idxi,'latitude'])
    lo2.append(tours.at[idxi,'longitude'])
    distance.append(distance1(latitude1[bikers.at[idxi1,"area"]],tours.at[idxi,'latitude'],longitude1[bikers.at[idxi1,"area"]],tours.at[idxi,'longitude']))
    wother.append(tours.at[idxi,'w_other'])
    w1.append(tours.at[idxi,'w1'])
    w2.append(tours.at[idxi,'w2'])
    w3.append(tours.at[idxi,'w3'])
    w4.append(tours.at[idxi,'w4'])
    w5.append(tours.at[idxi,'w5'])
    w6.append(tours.at[idxi,'w6'])
    w7.append(tours.at[idxi,'w7'])
    w8.append(tours.at[idxi,'w8'])
    w9.append(tours.at[idxi,'w9'])
    w10.append(tours.at[idxi,'w10'])
    w11.append(tours.at[idxi,'w11'])
    w12.append(tours.at[idxi,'w12'])
    w13.append(tours.at[idxi,'w13'])
    w14.append(tours.at[idxi,'w14'])
    w15.append(tours.at[idxi,'w15'])
    w16.append(tours.at[idxi,'w16'])
    w17.append(tours.at[idxi,'w17'])
    w18.append(tours.at[idxi,'w18'])
    w19.append(tours.at[idxi,'w19'])
    w20.append(tours.at[idxi,'w20'])
    w21.append(tours.at[idxi,'w21'])
    w22.append(tours.at[idxi,'w22'])
    w23.append(tours.at[idxi,'w23'])
    w24.append(tours.at[idxi,'w24'])
    w25.append(tours.at[idxi,'w25'])
    w26.append(tours.at[idxi,'w26'])
    w27.append(tours.at[idxi,'w27'])
    w28.append(tours.at[idxi,'w28'])
    w29.append(tours.at[idxi,'w29'])
    w30.append(tours.at[idxi,'w30'])
    w31.append(tours.at[idxi,'w31'])
    w32.append(tours.at[idxi,'w32'])
    w33.append(tours.at[idxi,'w33'])
    w34.append(tours.at[idxi,'w34'])
    w35.append(tours.at[idxi,'w35'])
    w36.append(tours.at[idxi,'w36'])
    w37.append(tours.at[idxi,'w37'])
    w38.append(tours.at[idxi,'w38'])
    w39.append(tours.at[idxi,'w39'])
    w40.append(tours.at[idxi,'w40'])
    w41.append(tours.at[idxi,'w41'])
    w42.append(tours.at[idxi,'w42'])
    w43.append(tours.at[idxi,'w43'])
    w44.append(tours.at[idxi,'w44'])
    w45.append(tours.at[idxi,'w45'])
    w46.append(tours.at[idxi,'w46'])
    w47.append(tours.at[idxi,'w47'])
    w48.append(tours.at[idxi,'w48'])
    w49.append(tours.at[idxi,'w49'])
    w50.append(tours.at[idxi,'w50'])
    w51.append(tours.at[idxi,'w51'])
    w52.append(tours.at[idxi,'w52'])
    w53.append(tours.at[idxi,'w53'])
    w54.append(tours.at[idxi,'w54'])
    w55.append(tours.at[idxi,'w55'])
    w56.append(tours.at[idxi,'w56'])
    w57.append(tours.at[idxi,'w57'])
    w58.append(tours.at[idxi,'w58'])
    w59.append(tours.at[idxi,'w59'])
    w60.append(tours.at[idxi,'w60'])
    w61.append(tours.at[idxi,'w61'])
    w62.append(tours.at[idxi,'w62'])
    w63.append(tours.at[idxi,'w63'])
    w64.append(tours.at[idxi,'w64'])
    w65.append(tours.at[idxi,'w65'])
    w66.append(tours.at[idxi,'w66'])
    w67.append(tours.at[idxi,'w67'])
    w68.append(tours.at[idxi,'w68'])
    w69.append(tours.at[idxi,'w69'])
    w70.append(tours.at[idxi,'w70'])
    w71.append(tours.at[idxi,'w71'])
    w72.append(tours.at[idxi,'w72'])
    w73.append(tours.at[idxi,'w73'])
    w74.append(tours.at[idxi,'w74'])
    w75.append(tours.at[idxi,'w75'])
    w76.append(tours.at[idxi,'w76'])
    w77.append(tours.at[idxi,'w77'])
    w78.append(tours.at[idxi,'w78'])
    w79.append(tours.at[idxi,'w79'])
    w80.append(tours.at[idxi,'w80'])
    w81.append(tours.at[idxi,'w81'])
    w82.append(tours.at[idxi,'w82'])
    w83.append(tours.at[idxi,'w83'])
    w84.append(tours.at[idxi,'w84'])
    w85.append(tours.at[idxi,'w85'])
    w86.append(tours.at[idxi,'w86'])
    w87.append(tours.at[idxi,'w87'])
    w88.append(tours.at[idxi,'w88'])
    w89.append(tours.at[idxi,'w89'])
    w90.append(tours.at[idxi,'w90'])
    w91.append(tours.at[idxi,'w91'])
    w92.append(tours.at[idxi,'w92'])
    w93.append(tours.at[idxi,'w93'])
    w94.append(tours.at[idxi,'w94'])
    w95.append(tours.at[idxi,'w95'])
    w96.append(tours.at[idxi,'w96'])
    w97.append(tours.at[idxi,'w97'])
    w98.append(tours.at[idxi,'w98'])
    w99.append(tours.at[idxi,'w99'])
    w100.append(tours.at[idxi,'w100'])
    stri=dic1[t]
    if stri.find(b) == -1 :
        notgoing.append(0)
    else :
        notgoing.append(1)
    stri=dic2[t]
    if stri.find(b) == -1 :
        maybe.append(0)
    else :
        maybe.append(1)
    stri=dic3[t]
    if stri.find(b) == -1 :
        going.append(0)
    else :
        going.append(1)
    stri=dic4[t]
    if stri.find(b) == -1 :
        invited.append(0)
    else :
        invited.append(1)

In [ ]:
#fgoing fnotgoing finvited fnotinvited data for ourtrain and ourtest

dic11 = dict(zip(tourco['tour_id'],tourco['going']))
dic22 = dict(zip(tourco['tour_id'],tourco['invited']))
dic33 = dict(zip(tourco['tour_id'],tourco['maybe']))
dic44 = dict(zip(tourco['tour_id'],tourco['not_going']))
dic = dict(zip(biken['biker_id'],biken['friends']))
fgoing = []
fnot_going = []
finvited = []
fmaybe = []

fgoing1 = []
fnot_going1 = []
finvited1 = []
fmaybe1 = []

for i in range(0,train.shape[0]):
    print(i)
    t = train.at[i,"tour_id"]
    b = train.at[i,"biker_id"]
    l = dic[b].split()
    st1 = dic11[t]
    st2 = dic22[t]
    st3 = dic33[t]
    st4 = dic44[t]
    s1 = 0
    s2 = 0
    s3 = 0
    s4 = 0
    for j in l:
        if st1.find(j) != -1:
            s1 = s1+1
        if st2.find(j) != -1:
            s2 = s2+1
        if st3.find(j) != -1:
            s3 = s3+1
        if st4.find(j) != -1:
            s4 = s4+1
    fgoing.append(s1)
    finvited.append(s2)
    fmaybe.append(s3)
    fnot_going.append(s4)

for i in range(0,test.shape[0]):
    print(i)
    t = test.at[i,"tour_id"]
    b = test.at[i,"biker_id"]
    l = dic[b].split()
    st1 = dic11[t]
    st2 = dic22[t]
    st3 = dic33[t]
    st4 = dic44[t]
    s1 = 0
    s2 = 0
    s3 = 0
    s4 = 0
    for j in l:
        if st1.find(j) != -1:
            s1 = s1+1
        if st2.find(j) != -1:
            s2 = s2+1
        if st3.find(j) != -1:
            s3 = s3+1
        if st4.find(j) != -1:
            s4 = s4+1
    fgoing1.append(s1)
    finvited1.append(s2)
    fmaybe1.append(s3)
    fnot_going1.append(s4)

In [ ]:
ourtrain=pd.DataFrame()
ourtrain['born']=born
ourtrain['ifri']=ifri
ourtrain['male']=male
ourtrain['female']=female
ourtrain['timegap']=timegap
ourtrain['la1']=la1
ourtrain['lo1']=lo1
ourtrain['la2']=la2
ourtrain['lo2']=lo2
ourtrain['distance']=distance
ourtrain['invited']=invited 
ourtrain['notgoing']=notgoing 
ourtrain['maybe']=maybe 
ourtrain['going']=going
ourtrain['wother']=wother
ourtrain['w1']=w1
ourtrain['w2']=w2
ourtrain['w3']=w3
ourtrain['w4']=w4
ourtrain['w5']=w5
ourtrain['w6']=w6
ourtrain['w7']=w7
ourtrain['w8']=w8
ourtrain['w9']=w9
ourtrain['w10']=w10
ourtrain['w11']=w11
ourtrain['w12']=w12
ourtrain['w13']=w13
ourtrain['w14']=w14
ourtrain['w15']=w15
ourtrain['w16']=w16
ourtrain['w17']=w17
ourtrain['w18']=w18
ourtrain['w19']=w19
ourtrain['w20']=w20
ourtrain['w21']=w21
ourtrain['w22']=w22
ourtrain['w23']=w23
ourtrain['w24']=w24
ourtrain['w25']=w25
ourtrain['w26']=w26
ourtrain['w27']=w27
ourtrain['w28']=w28
ourtrain['w29']=w29
ourtrain['w30']=w30
ourtrain['w31']=w31
ourtrain['w32']=w32
ourtrain['w33']=w33
ourtrain['w34']=w34
ourtrain['w35']=w35
ourtrain['w36']=w36
ourtrain['w37']=w37
ourtrain['w38']=w38
ourtrain['w39']=w39
ourtrain['w40']=w40
ourtrain['w41']=w41
ourtrain['w42']=w42
ourtrain['w43']=w43
ourtrain['w44']=w44
ourtrain['w45']=w45
ourtrain['w46']=w46
ourtrain['w47']=w47
ourtrain['w48']=w48
ourtrain['w49']=w49
ourtrain['w50']=w50
ourtrain['w51']=w51
ourtrain['w52']=w52
ourtrain['w53']=w53
ourtrain['w54']=w54
ourtrain['w55']=w55
ourtrain['w56']=w56
ourtrain['w57']=w57
ourtrain['w58']=w58
ourtrain['w59']=w59
ourtrain['w60']=w60
ourtrain['w61']=w61
ourtrain['w62']=w62
ourtrain['w63']=w63
ourtrain['w64']=w64
ourtrain['w65']=w65
ourtrain['w66']=w66
ourtrain['w67']=w67
ourtrain['w68']=w68
ourtrain['w69']=w69
ourtrain['w70']=w70
ourtrain['w71']=w71
ourtrain['w72']=w72
ourtrain['w73']=w73
ourtrain['w74']=w74
ourtrain['w75']=w75
ourtrain['w76']=w76
ourtrain['w77']=w77
ourtrain['w78']=w78
ourtrain['w79']=w79
ourtrain['w80']=w80
ourtrain['w81']=w81
ourtrain['w82']=w82
ourtrain['w83']=w83
ourtrain['w84']=w84
ourtrain['w85']=w85
ourtrain['w86']=w86
ourtrain['w87']=w87
ourtrain['w88']=w88
ourtrain['w89']=w89
ourtrain['w90']=w90
ourtrain['w91']=w91
ourtrain['w92']=w92
ourtrain['w93']=w93
ourtrain['w94']=w94
ourtrain['w95']=w95
ourtrain['w96']=w96
ourtrain['w97']=w97
ourtrain['w98']=w98
ourtrain['w99']=w99
ourtrain['w100']=w100
ourtrain['friends_invited']=finvited 
ourtrain['friends_notgoing']=fnot_going 
ourtrain['friends_maybe']=fmaybe 
ourtrain['friends_going']=fgoing

In [ ]:
ourtrain

In [ ]:
#y for multiclassification
y=[]
like=train["like"]
dislike=train["dislike"]
for i in range(train.shape[0]):
    if dislike[i]==1 :
        y.append(0)
    elif dislike[i]==0 and like[i]==0:
        y.append(5)
    else :
        y.append(10)
#y for binary classification
y1=[]
for i in range(train.shape[0]):
    if dislike[i]==1 :
        y1.append(0)
    elif dislike[i]==0 and like[i]==0:
        y1.append(0)
    else :
        y1.append(1)


In [ ]:
# w's,la2,lo2,lo1,la1,friends invited,maybe,going,not going,distance  for ourtest
w1=[]
w2=[]
w3=[]
w4=[]
w5=[]
w6=[]
w7=[]
w8=[]
w9=[]
w10=[]
w11=[]
w12=[]
w13=[]
w14=[]
w15=[]
w16=[]
w17=[]
w18=[]
w19=[]
w20=[]
w21=[]
w22=[]
w23=[]
w24=[]
w25=[]
w26=[]
w27=[]
w28=[]
w29=[]
w30=[]
w31=[]
w32=[]
w33=[]
w34=[]
w35=[]
w36=[]
w37=[]
w38=[]
w39=[]
w40=[]
w41=[]
w42=[]
w43=[]
w44=[]
w45=[]
w46=[]
w47=[]
w48=[]
w49=[]
w50=[]
w51=[]
w52=[]
w53=[]
w54=[]
w55=[]
w56=[]
w57=[]
w58=[]
w59=[]
w60=[]
w61=[]
w62=[]
w63=[]
w64=[]
w65=[]
w66=[]
w67=[]
w68=[]
w69=[]
w70=[]
w71=[]
w72=[]
w73=[]
w74=[]
w75=[]
w76=[]
w77=[]
w78=[]
w79=[]
w80=[]
w81=[]
w82=[]
w83=[]
w84=[]
w85=[]
w86=[]
w87=[]
w88=[]
w89=[]
w90=[]
w91=[]
w92=[]
w93=[]
w94=[]
w95=[]
w96=[]
w97=[]
w98=[]
w99=[]
w100=[]
wother = []
la2 = []
lo2 = []
la1 = []
lo1 = []
invited = []
notgoing = []
maybe = []
going = []
distance = []

for i in range(test.shape[0]):
    print(i)
    b = test.at[i,'biker_id']
    l = bikers.index[bikers['biker_id'] == b].tolist()
    idxi1 = l[0]
    la1.append(bikeloc.at[idxi1,"lat"])
    lo1.append(bikeloc.at[idxi1,"long"])
    t = test.at[i,'tour_id']
    l = tours.index[tours['tour_id'] == t].tolist()
    idxi = l[0]
    la2.append(tours.at[idxi,'latitude'])
    lo2.append(tours.at[idxi,'longitude'])
    distance.append(distance1(latitude1[bikers.at[idxi1,"area"]],tours.at[idxi,'latitude'],longitude1[bikers.at[idxi1,"area"]],tours.at[idxi,'longitude']))
    wother.append(tours.at[idxi,'w_other'])
    w1.append(tours.at[idxi,'w1'])
    w2.append(tours.at[idxi,'w2'])
    w3.append(tours.at[idxi,'w3'])
    w4.append(tours.at[idxi,'w4'])
    w5.append(tours.at[idxi,'w5'])
    w6.append(tours.at[idxi,'w6'])
    w7.append(tours.at[idxi,'w7'])
    w8.append(tours.at[idxi,'w8'])
    w9.append(tours.at[idxi,'w9'])
    w10.append(tours.at[idxi,'w10'])
    w11.append(tours.at[idxi,'w11'])
    w12.append(tours.at[idxi,'w12'])
    w13.append(tours.at[idxi,'w13'])
    w14.append(tours.at[idxi,'w14'])
    w15.append(tours.at[idxi,'w15'])
    w16.append(tours.at[idxi,'w16'])
    w17.append(tours.at[idxi,'w17'])
    w18.append(tours.at[idxi,'w18'])
    w19.append(tours.at[idxi,'w19'])
    w20.append(tours.at[idxi,'w20'])
    w21.append(tours.at[idxi,'w21'])
    w22.append(tours.at[idxi,'w22'])
    w23.append(tours.at[idxi,'w23'])
    w24.append(tours.at[idxi,'w24'])
    w25.append(tours.at[idxi,'w25'])
    w26.append(tours.at[idxi,'w26'])
    w27.append(tours.at[idxi,'w27'])
    w28.append(tours.at[idxi,'w28'])
    w29.append(tours.at[idxi,'w29'])
    w30.append(tours.at[idxi,'w30'])
    w31.append(tours.at[idxi,'w31'])
    w32.append(tours.at[idxi,'w32'])
    w33.append(tours.at[idxi,'w33'])
    w34.append(tours.at[idxi,'w34'])
    w35.append(tours.at[idxi,'w35'])
    w36.append(tours.at[idxi,'w36'])
    w37.append(tours.at[idxi,'w37'])
    w38.append(tours.at[idxi,'w38'])
    w39.append(tours.at[idxi,'w39'])
    w40.append(tours.at[idxi,'w40'])
    w41.append(tours.at[idxi,'w41'])
    w42.append(tours.at[idxi,'w42'])
    w43.append(tours.at[idxi,'w43'])
    w44.append(tours.at[idxi,'w44'])
    w45.append(tours.at[idxi,'w45'])
    w46.append(tours.at[idxi,'w46'])
    w47.append(tours.at[idxi,'w47'])
    w48.append(tours.at[idxi,'w48'])
    w49.append(tours.at[idxi,'w49'])
    w50.append(tours.at[idxi,'w50'])
    w51.append(tours.at[idxi,'w51'])
    w52.append(tours.at[idxi,'w52'])
    w53.append(tours.at[idxi,'w53'])
    w54.append(tours.at[idxi,'w54'])
    w55.append(tours.at[idxi,'w55'])
    w56.append(tours.at[idxi,'w56'])
    w57.append(tours.at[idxi,'w57'])
    w58.append(tours.at[idxi,'w58'])
    w59.append(tours.at[idxi,'w59'])
    w60.append(tours.at[idxi,'w60'])
    w61.append(tours.at[idxi,'w61'])
    w62.append(tours.at[idxi,'w62'])
    w63.append(tours.at[idxi,'w63'])
    w64.append(tours.at[idxi,'w64'])
    w65.append(tours.at[idxi,'w65'])
    w66.append(tours.at[idxi,'w66'])
    w67.append(tours.at[idxi,'w67'])
    w68.append(tours.at[idxi,'w68'])
    w69.append(tours.at[idxi,'w69'])
    w70.append(tours.at[idxi,'w70'])
    w71.append(tours.at[idxi,'w71'])
    w72.append(tours.at[idxi,'w72'])
    w73.append(tours.at[idxi,'w73'])
    w74.append(tours.at[idxi,'w74'])
    w75.append(tours.at[idxi,'w75'])
    w76.append(tours.at[idxi,'w76'])
    w77.append(tours.at[idxi,'w77'])
    w78.append(tours.at[idxi,'w78'])
    w79.append(tours.at[idxi,'w79'])
    w80.append(tours.at[idxi,'w80'])
    w81.append(tours.at[idxi,'w81'])
    w82.append(tours.at[idxi,'w82'])
    w83.append(tours.at[idxi,'w83'])
    w84.append(tours.at[idxi,'w84'])
    w85.append(tours.at[idxi,'w85'])
    w86.append(tours.at[idxi,'w86'])
    w87.append(tours.at[idxi,'w87'])
    w88.append(tours.at[idxi,'w88'])
    w89.append(tours.at[idxi,'w89'])
    w90.append(tours.at[idxi,'w90'])
    w91.append(tours.at[idxi,'w91'])
    w92.append(tours.at[idxi,'w92'])
    w93.append(tours.at[idxi,'w93'])
    w94.append(tours.at[idxi,'w94'])
    w95.append(tours.at[idxi,'w95'])
    w96.append(tours.at[idxi,'w96'])
    w97.append(tours.at[idxi,'w97'])
    w98.append(tours.at[idxi,'w98'])
    w99.append(tours.at[idxi,'w99'])
    w100.append(tours.at[idxi,'w100'])
    stri=dic1[t]
    if stri.find(b) == -1 :
        notgoing.append(0)
    else :
        notgoing.append(1)
    stri=dic2[t]
    if stri.find(b) == -1 :
        maybe.append(0)
    else :
        maybe.append(1)
    stri=dic3[t]
    if stri.find(b) == -1 :
        going.append(0)
    else :
        going.append(1)
    stri=dic4[t]
    if stri.find(b) == -1 :
        invited.append(0)
    else :
        invited.append(1)

In [ ]:
#categorical data area,location,language for ourtrain,ourtest
bikers['area'].fillna('NO',inplace= True)
labelencoder = LabelEncoder()
bikers['area'] = labelencoder.fit_transform(bikers['area'])
dicarea=dict(zip(bikers['biker_id'],bikers['area']))

labelencoder = LabelEncoder()
bikers['location_id'] = labelencoder.fit_transform(bikers['location_id'])
diclocation=dict(zip(bikers['biker_id'],bikers['location_id']))

labelencoder = LabelEncoder()
bikers['language_id'] = labelencoder.fit_transform(bikers['language_id'])
diclanguage=dict(zip(bikers['biker_id'],bikers['language_id']))

areazzz=[] 
locationzzz=[]    
languagezzz=[]
areazzz1=[] 
locationzzz1=[]    
languagezzz1=[]
for i in range(train.shape[0]):
    areazzz.append(dicarea[train.at[i,'biker_id']])
    locationzzz.append(diclocation[train.at[i,'biker_id']])
    languagezzz.append(diclanguage[train.at[i,'biker_id']])
for i in range(test.shape[0]):
    areazzz1.append(dicarea[test.at[i,'biker_id']])
    locationzzz1.append(diclocation[test.at[i,'biker_id']])
    languagezzz1.append(diclanguage[test.at[i,'biker_id']])

In [ ]:
ourtrain['areazzz']=areazzz
ourtrain['locationzzz']=locationzzz
ourtrain['languagezzz']=languagezzz

In [ ]:
ourtest=pd.DataFrame()
ourtest['born']=born1
ourtest['ifri']=ifri1
ourtest['male']=male1
ourtest['female']=female1
ourtest['timegap']=timegap1
ourtest['la1']=la1
ourtest['lo1']=lo1
ourtest['la2']=la2
ourtest['lo2']=lo2
ourtest['distance']=distance
ourtest['invited']=invited 
ourtest['notgoing']=notgoing 
ourtest['maybe']=maybe 
ourtest['going']=going
ourtest['wother']=wother
ourtest['w1']=w1
ourtest['w2']=w2
ourtest['w3']=w3
ourtest['w4']=w4
ourtest['w5']=w5
ourtest['w6']=w6
ourtest['w7']=w7
ourtest['w8']=w8
ourtest['w9']=w9
ourtest['w10']=w10
ourtest['w11']=w11
ourtest['w12']=w12
ourtest['w13']=w13
ourtest['w14']=w14
ourtest['w15']=w15
ourtest['w16']=w16
ourtest['w17']=w17
ourtest['w18']=w18
ourtest['w19']=w19
ourtest['w20']=w20
ourtest['w21']=w21
ourtest['w22']=w22
ourtest['w23']=w23
ourtest['w24']=w24
ourtest['w25']=w25
ourtest['w26']=w26
ourtest['w27']=w27
ourtest['w28']=w28
ourtest['w29']=w29
ourtest['w30']=w30
ourtest['w31']=w31
ourtest['w32']=w32
ourtest['w33']=w33
ourtest['w34']=w34
ourtest['w35']=w35
ourtest['w36']=w36
ourtest['w37']=w37
ourtest['w38']=w38
ourtest['w39']=w39
ourtest['w40']=w40
ourtest['w41']=w41
ourtest['w42']=w42
ourtest['w43']=w43
ourtest['w44']=w44
ourtest['w45']=w45
ourtest['w46']=w46
ourtest['w47']=w47
ourtest['w48']=w48
ourtest['w49']=w49
ourtest['w50']=w50
ourtest['w51']=w51
ourtest['w52']=w52
ourtest['w53']=w53
ourtest['w54']=w54
ourtest['w55']=w55
ourtest['w56']=w56
ourtest['w57']=w57
ourtest['w58']=w58
ourtest['w59']=w59
ourtest['w60']=w60
ourtest['w61']=w61
ourtest['w62']=w62
ourtest['w63']=w63
ourtest['w64']=w64
ourtest['w65']=w65
ourtest['w66']=w66
ourtest['w67']=w67
ourtest['w68']=w68
ourtest['w69']=w69
ourtest['w70']=w70
ourtest['w71']=w71
ourtest['w72']=w72
ourtest['w73']=w73
ourtest['w74']=w74
ourtest['w75']=w75
ourtest['w76']=w76
ourtest['w77']=w77
ourtest['w78']=w78
ourtest['w79']=w79
ourtest['w80']=w80
ourtest['w81']=w81
ourtest['w82']=w82
ourtest['w83']=w83
ourtest['w84']=w84
ourtest['w85']=w85
ourtest['w86']=w86
ourtest['w87']=w87
ourtest['w88']=w88
ourtest['w89']=w89
ourtest['w90']=w90
ourtest['w91']=w91
ourtest['w92']=w92
ourtest['w93']=w93
ourtest['w94']=w94
ourtest['w95']=w95
ourtest['w96']=w96
ourtest['w97']=w97
ourtest['w98']=w98
ourtest['w99']=w99
ourtest['w100']=w100
ourtest['friends_invited']=finvited1
ourtest['friends_notgoing']=fnot_going1 
ourtest['friends_maybe']=fmaybe1 
ourtest['friends_going']=fgoing1
ourtest['areazzz']=areazzz1
ourtest['locationzzz']=locationzzz1
ourtest['languagezzz']=languagezzz1

In [ ]:
categorical_features = [c for c, col in enumerate(ourtrain.columns) if 'zzz' in col]

train_data = lightgbm.Dataset(ourtrain, label=y, categorical_feature=categorical_features)
parameters = {
    'application': 'binary',
    'objective': 'multiclass',
    'num_leaves': 80,
    'num_rounds': 50,
    'num_class' : 11
}


In [ ]:
model = lightgbm.train(parameters,train_data)

In [ ]:
lightgbm.plot_importance(model,max_num_features=10)

In [ ]:
xx=ourtest
ypred1=model.predict(xx)
print(ypred1)
ypred=[]
for i in range(ypred1.shape[0]):
    s=ypred1[i]
    su=0
    for j in range(len(s)):
        su+=(j)*s[j]
    ypred.append(su)
print(ypred)

In [ ]:
bikers_df = test.drop_duplicates(subset="biker_id")
ypredpd = pd.DataFrame(columns=["y"])
ypredpd['y'] = ypred
bikers_set = np.array(bikers_df["biker_id"])
bikers = []
tours = []
for biker in bikers_set:
    idx = np.where(biker==test["biker_id"]) 
    X = list(test["tour_id"].loc[idx]) 
    Y = list(ypredpd["y"].loc[idx])
    tour = [x for _,x in sorted(zip(Y,X))]
    tour.reverse()
    tour = " ".join(tour) # list to space delimited string
    bikers.append(biker)
    tours.append(tour)

In [ ]:
sample_submission =pd.DataFrame(columns=["biker_id","tour_id"])
sample_submission["biker_id"] = bikers
sample_submission["tour_id"] = tours
sample_submission.to_csv("cs18b039_1.csv",index=False) 
sample_submission.head(4)


In [ ]:
train_data = lightgbm.Dataset(ourtrain, label=y1, categorical_feature=categorical_features)
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'num_leaves': 80,
    'num_rounds': 50
}


In [ ]:
model = lightgbm.train(parameters,train_data)

In [ ]:
xx=ourtest
ypred=model.predict(xx)
print(ypred)

In [ ]:
bikers_df = test.drop_duplicates(subset="biker_id")
ypredpd = pd.DataFrame(columns=["y"])
ypredpd['y'] = ypred
bikers_set = np.array(bikers_df["biker_id"])
bikers = []
tours = []
for biker in bikers_set:
    idx = np.where(biker==test["biker_id"]) 
    X = list(test["tour_id"].loc[idx]) 
    Y = list(ypredpd["y"].loc[idx])
    tour = [x for _,x in sorted(zip(Y,X))]
    tour.reverse()
    tour = " ".join(tour) # list to space delimited string
    bikers.append(biker)
    tours.append(tour)

In [ ]:
sample_submission =pd.DataFrame(columns=["biker_id","tour_id"])
sample_submission["biker_id"] = bikers
sample_submission["tour_id"] = tours
sample_submission.to_csv("cs18b039_2.csv",index=False) 
sample_submission.head(4)
